In [1]:
import datetime
import Model
import numpy as np
import pandas as pd
import math
from multiprocessing import Pool
import os
import warnings
import time
import matplotlib.pyplot as plt
from importlib import reload

reload(Model)
warnings.simplefilter('ignore', FutureWarning)

# positive numbers, start should be lower than end
# a rebalance index represents the month before the rebalancing takes place
# so returns are calculated starting at rebalanceIndex + 1
def rebalanceIndexes(startIndex, endIndex):
    indexes = list(range(startIndex, endIndex, 3))
    return (indexes)


# get ln returns for an equally balanced portfolio of stocks
def getReturns(portfolio, index, length):
	returns = 0
	print(portfolio)
	print(-1 * index + 1)
	print(-1 * index + length)
	nancount = 0
	for stock in portfolio:
		indReturn = Model.rateOfReturn(Model.retrieveData(stock, 'Last Price', -1 * index + 1, -1 * index + length, []))
		print(stock + ": " + str(indReturn))
		if (not math.isnan(indReturn)):
			returns += indReturn
		else:
			nancount += 1
	if len(portfolio) - nancount == 0:
		total = 0
	else:
		total = returns/(len(portfolio)-nancount)
	print("Start Date: " + str(Model.convertIndexToDate(-1 * index + 1)))
	print("Total Return: " +str(total))
	return (total)


# make a portfolio with predicted probabilities higher than a hardcoded threshold
def makePortfolio(treeTuple):
	featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
	addedStocks, probabilities = Model.predict_probabilities(treeTuple[1], startIndex = -1 * treeTuple[0] - 11, endIndex = -1 * treeTuple[0], features = featureList, sector = "Consumer Discretionary")
	probabilityThreshold = 0.8
	stockTuples = zip(addedStocks, probabilities)
	stockTuples = list(filter(lambda x: x[1][1] > probabilityThreshold, stockTuples))
	if len(stockTuples) == 0:
		print("No portfolio, probabilities lower than threshold of " + str(probabilityThreshold))
		return 0
	stocks, probabilities = zip(*stockTuples)
	return(getReturns(stocks, treeTuple[0], 3))

In [2]:
indexes = []
for i in rebalanceIndexes(4,56):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

18
[array([-219, -216, -213, -210, -207, -204, -201, -198, -195, -192, -189,
       -186, -183, -180, -177, -174, -171, -168, -165, -162, -159, -156,
       -153, -150, -147, -144, -141, -138, -135, -132, -129, -126, -123,
       -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,  -90,
        -87,  -84,  -81,  -78,  -75,  -72,  -69,  -66,  -63,  -60,  -57,
        -54,  -51,  -48,  -45,  -42,  -39,  -36,  -33,  -30,  -27,  -24,
        -21]), array([-222, -219, -216, -213, -210, -207, -204, -201, -198, -195, -192,
       -189, -186, -183, -180, -177, -174, -171, -168, -165, -162, -159,
       -156, -153, -150, -147, -144, -141, -138, -135, -132, -129, -126,
       -123, -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,
        -90,  -87,  -84,  -81,  -78,  -75,  -72,  -69,  -66,  -63,  -60,
        -57,  -54,  -51,  -48,  -45,  -42,  -39,  -36,  -33,  -30,  -27,
        -24]), array([-225, -222, -219, -216, -213, -210, -207, -204, -201, -198, -195,
       -192, -189

In [3]:
print("# of multiprocess cpus: " + str(os.cpu_count()))
sector = "Consumer Discretionary"
featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
forestList = []
for ind in indexes:
	randForest = Model.buildWithIndexesTripleClass(modelType = Model.randomForestClassifier, indexes = ind, target= 'Rate of Return', features = featureList, featureLength = 12,\
									targetLength = 3, sector = sector, percentileTarget = 90, percentileAvoid = 10, verbose = True)
	forestList.append(randForest)
print(forestList)

# of multiprocess cpus: 8
Finished data retrieval, starting model training. Time taken: 125.98543810844421 seconds.
Finished fitting. Time taken: 23.425883054733276 seconds.
Finished data retrieval, starting model training. Time taken: 117.17932224273682 seconds.
Finished fitting. Time taken: 23.369773864746094 seconds.
Finished data retrieval, starting model training. Time taken: 147.02545404434204 seconds.
Finished fitting. Time taken: 23.65790891647339 seconds.
Finished data retrieval, starting model training. Time taken: 152.43960690498352 seconds.
Finished fitting. Time taken: 26.623984813690186 seconds.
Finished data retrieval, starting model training. Time taken: 118.300705909729 seconds.
Finished fitting. Time taken: 23.866416215896606 seconds.
Finished data retrieval, starting model training. Time taken: 118.92869710922241 seconds.
Finished fitting. Time taken: 22.31186008453369 seconds.
Finished data retrieval, starting model training. Time taken: 444.9844319820404 seconds.
F

In [4]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsList = pool.map(makePortfolio, zip(rebalanceIndexes(4,56), forestList))
print(returnsList)

# of multiprocess cpus: 8
('CTRN', 'ZAGG')
Total Return: -0.14756756020368833
-3
-1
CTRN: -0.09256095734127534
ZAGG: -0.20257416306610132
Start Date: 2017-11-30
('CPLA', 'CTRN', 'DAN', 'ERI', 'EXPR', 'FIVE', 'OSTK', 'RH', 'SHAK', 'TLYS', 'TPH', 'ZAGG')
ERI: 0.11099677597202451
-6
-4
CPLA: 0.19008643219168064
CTRN: 0.1825053972209827
DAN: 0.23643250920518266
EXPR: 0.06090161734011157
FIVE: 0.14966602995367229
OSTK: 0.7376979775453192
RH: 0.6532508812672555
SHAK: 0.20512975002250577
TLYS: 0.07857443263262054
TPH: 0.3282528580276769
ZAGG: 0.21677410302873001
Start Date: 2017-08-31
Total Return: 0.26252239703398017
('CROX', 'ERI', 'SNI', 'SNOW', 'W', 'ZAGG')
-9
-7
CROX: 0.15058860167634558
ERI: -0.01701134582653685
SNI: 0.27762716047329583
SNOW: 0.16129240795833066
W: 0.19298727117428527
ZAGG: 0.011904902506318038
Start Date: 2017-05-31
Total Return: 0.12956483299367308
('BPI', 'ILG', 'IRBT', 'LOPE', 'OLLI', 'RH', 'W')
Total Return: 0.270624932731843
-12
-10
BPI: 0.2682009368799583
ILG: 0.